# Identify generic genes and pathways

Studies have found that some genes are more likely to be differentially expressed even across a wide range of experimental designs. These generic genes and subsequent pathways are not necessarily specific to the biological process being studied but instead represent a more systematic change.

We have developed an approach, outlined below, to automatically identify these generic genes and pathways. We have validated this simulation approach can identify generic genes and pathways in the analysis notebooks: [human_general_analysis](../human_general_analysis/) and [human_cancer_analysis](../human_cancer_analysis/). Here

This notebook applies this approach to identify generic genes and pathways in the pseudomonas compendium.

**Steps to identify generic genes:**
1. Simulates N gene expression experiments using [ponyo](https://github.com/ajlee21/ponyo)
2. Perform DE analysis to get association statistics for each gene

In this case the DE analysis is based on the experimental design of the template experiment, described in the previous [notebook](1_process_pseudomonas_data.ipynb).
The template experiment is [GEOD-33245](https://www.ebi.ac.uk/arrayexpress/experiments/E-GEOD-33245/?s_sortby=col_8&s_sortorder=ascending), which contains multiple different comparisons including WT vs *crc* mutants, WT vs *cbr* mutants in different conditions. So the DE analysis is comparing WT vs mutant.

3. For each gene, aggregate statistics across all simulated experiments
4. Rank genes based on this aggregated statistic

**Steps to identify generic gene sets (pathways):**
1. Using the same simulated experiments from above, perform GSEA analysis. This analysis will determine whether the genes contained in a gene set are clustered towards the beginning or the end of the ranked list of genes, where genes are ranked by log fold change, indicating a correlation with change in expression.
2. For each gene set (pathway), aggregate statistics across all simulated experiments
3. Rank gene sets based on this aggregated statistic

In [1]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2
%matplotlib inline

import os
import sys
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import scipy.stats as ss

from rpy2.robjects import pandas2ri
from ponyo import utils, simulate_expression_data
from generic_expression_patterns_modules import process, stats, ranking

pandas2ri.activate()

np.random.seed(123)

/home/alexandra/anaconda3/envs/generic_expression_new/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
Using TensorFlow backend.


In [2]:
# Read in config variables
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

config_filename = os.path.abspath(
    os.path.join(base_dir, "configs", "config_pseudomonas_33245.tsv")
)
params = utils.read_config(config_filename)

In [3]:
# Load params
local_dir = params["local_dir"]
dataset_name = params["dataset_name"]
NN_architecture = params["NN_architecture"]
latent_dim = params["latent_dim"]
num_runs = params["num_simulated"]
project_id = params["project_id"]
metadata_col_id = params["metadata_colname"]
raw_template_filename = params["raw_template_filename"]
processed_template_filename = params["processed_template_filename"]
normalized_compendium_filename = params["normalized_compendium_filename"]
scaler_filename = params["scaler_filename"]
col_to_rank_genes = params["rank_genes_by"]
col_to_rank_pathways = params["rank_pathways_by"]
statistic = params["gsea_statistic"]
logFC_name = params["DE_logFC_name"]
pvalue_name = params["DE_pvalue_name"]

# Load metadata file with grouping assignments for samples
sample_id_metadata_filename = os.path.join(
    base_dir, dataset_name, "data", "metadata", f"{project_id}_process_samples.tsv"
)

# Load metadata file with grouping assignments for samples
metadata_filename = os.path.join(
    base_dir, dataset_name, "data", "metadata", f"{project_id}_groups.tsv"
)

# Load metadata file with mapping between experiments and associated samples
metadata_simulate_filename = os.path.join(
    base_dir, dataset_name, "data", "metadata", "sample_annotations.tsv"
)
metadata_delimiter = "\t"
experiment_id_colname = "experiment"

# Load pickled file
scaler = pickle.load(open(scaler_filename, "rb"))

# Percentile threshold to identify generic genes
percentile_threshold = 80.0

/home/alexandra/anaconda3/envs/generic_expression_new/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [4]:
# Output files
gene_summary_filename = os.path.join(
    base_dir, dataset_name, f"generic_gene_summary_{project_id}.tsv"
)
pathway_summary_filename = os.path.join(
    base_dir, dataset_name, f"generic_pathway_summary_{project_id}.tsv"
)

### Simulate experiments using selected template experiment
Workflow:

1. Get the gene expression data for the selected template experiment
2. Encode this experiment into a latent space using the trained VAE model
3. Linearly shift the encoded template experiment in the latent space
4. Decode the samples. This results in a new experiment
5. Repeat steps 1-4 to get multiple simulated experiments

In [5]:
# Simulate multiple experiments
# This step creates the following files in "<local_dir>/pseudo_experiment/" directory:
#   - selected_simulated_data_SRP012656_<n>.txt
#   - selected_simulated_encoded_data_SRP012656_<n>.txt
#   - template_normalized_data_SRP012656_test.txt
# in which "<n>" is an integer in the range of [0, num_runs-1]
os.makedirs(os.path.join(local_dir, "pseudo_experiment"), exist_ok=True)
simulate_expression_data.shift_template_experiment(
    normalized_compendium_filename,
    NN_architecture,
    latent_dim,
    dataset_name,
    scaler,
    metadata_simulate_filename,
    metadata_delimiter,
    experiment_id_colname,
    metadata_col_id,
    project_id,
    local_dir,
    base_dir,
    num_runs,
)

'# Simulate multiple experiments\n# This step creates the following files in "<local_dir>/pseudo_experiment/" directory:\n#   - selected_simulated_data_SRP012656_<n>.txt\n#   - selected_simulated_encoded_data_SRP012656_<n>.txt\n#   - template_normalized_data_SRP012656_test.txt\n# in which "<n>" is an integer in the range of [0, num_runs-1]\nos.makedirs(os.path.join(local_dir, "pseudo_experiment"), exist_ok=True)\nsimulate_expression_data.shift_template_experiment(\n    normalized_compendium_filename,\n    NN_architecture,\n    latent_dim,\n    dataset_name,\n    scaler,\n    metadata_simulate_filename,\n    metadata_delimiter,\n    experiment_id_colname,\n    metadata_col_id,\n    project_id,\n    local_dir,\n    base_dir,\n    num_runs,\n)'

### Process template and simulated data

* Remove samples not required for comparison.
* Make sure ordering of samples matches metadata for proper comparison

In [6]:
if not os.path.exists(sample_id_metadata_filename):
    sample_id_metadata_filename = None

stats.process_samples_for_limma(
    raw_template_filename,
    metadata_filename,
    processed_template_filename,
    sample_id_metadata_filename,
)

for i in range(num_runs):
    simulated_filename = os.path.join(
        local_dir, "pseudo_experiment", f"selected_simulated_data_{project_id}_{i}.txt"
    )
    stats.process_samples_for_limma(
        simulated_filename,
        metadata_filename,
        None,
        sample_id_metadata_filename,
    )

'if not os.path.exists(sample_id_metadata_filename):\n    sample_id_metadata_filename = None\n\nstats.process_samples_for_limma(\n    raw_template_filename,\n    metadata_filename,\n    processed_template_filename,\n    sample_id_metadata_filename,\n)\n\nfor i in range(num_runs):\n    simulated_filename = os.path.join(\n        local_dir, "pseudo_experiment", f"selected_simulated_data_{project_id}_{i}.txt"\n    )\n    stats.process_samples_for_limma(\n        simulated_filename,\n        metadata_filename,\n        None,\n        sample_id_metadata_filename,\n    )'

In [7]:
# Quick check
template_data = pd.read_csv(
    processed_template_filename, header=0, index_col=0, sep="\t"
)

assert template_data.shape[0] == 4

### Differential expression analysis

In [8]:
# Create subdirectory: "<local_dir>/DE_stats/"
os.makedirs(os.path.join(local_dir, "DE_stats"), exist_ok=True)

In [9]:
%%R -i metadata_filename -i project_id -i processed_template_filename -i local_dir -i base_dir

source(paste0(base_dir, '/generic_expression_patterns_modules/DE_analysis.R'))

get_DE_stats_limma(metadata_filename,
                   project_id,
                   processed_template_filename,
                   "template",
                   local_dir,
                   "real")

R[write to console]: Loading required package: S4Vectors

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following object is masked from ‘package:limma’:

    plotMA


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply

[1] 15


In [10]:
%%R -i metadata_filename -i project_id -i base_dir -i local_dir -i num_runs -o num_sign_DEGs_simulated

source(paste0(base_dir,'/generic_expression_patterns_modules/DE_analysis.R'))

num_sign_DEGs_simulated <- c()

for (i in 0:(num_runs-1)){
    simulated_data_filename <- paste(
        local_dir,
        "pseudo_experiment/selected_simulated_data_",
        project_id,
        "_",
        i,
        ".txt",
        sep=""
    )

    run_output <- get_DE_stats_limma(
        metadata_filename,
        project_id,
        simulated_data_filename,
        "simulated",
        local_dir,
        i
    )
    num_sign_DEGs_simulated <- c(num_sign_DEGs_simulated, run_output)
}

### Rank genes

In [11]:
analysis_type = "DE"
template_DE_stats_filename = os.path.join(
    local_dir, "DE_stats", f"DE_stats_template_data_{project_id}_real.txt"
)
template_DE_stats, simulated_DE_summary_stats = ranking.process_and_rank_genes_pathways(
    template_DE_stats_filename,
    local_dir,
    num_runs,
    project_id,
    analysis_type,
    col_to_rank_genes,
    logFC_name,
    pvalue_name,
)

/home/alexandra/anaconda3/envs/generic_expression_new/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


### Gene summary table

In [12]:
summary_gene_ranks = ranking.generate_summary_table(
    template_DE_stats_filename,
    template_DE_stats,
    simulated_DE_summary_stats,
    col_to_rank_genes,
    local_dir,
    "gene",
    params,
)

summary_gene_ranks.sort_values(by="Z score", ascending=False).head()

,Gene ID,Adj P-value (Real),Rank (Real),abs(logFC) (Real),logFC (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(logFC) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
PA4726,PA4726,0.000010,5549.0,4.530917,-4.530917,0.846422,4845.5,87.319755,0.0,0.16708,0.101184,25,43.127565
PA1123,PA1123,0.002578,5537.0,2.175900,-2.175900,0.682483,4391.5,79.136626,0.0,0.16190,0.106530,25,18.905485
PA4620,PA4620,0.001107,5512.0,1.611311,1.611311,0.842271,2220.5,40.005407,0.0,0.11220,0.092593,25,16.190288
PA1217,PA1217,0.001039,5531.0,1.941753,1.941753,0.850699,1612.0,29.037491,0.0,0.11886,0.120587,25,15.116766
PA1216,PA1216,0.000223,5545.0,2.600594,2.600594,0.832988,3823.5,68.898702,0.0,0.19604,0.159747,25,15.052271


In [13]:
# Add gene name as column to summary dataframe
summary_gene_ranks = ranking.add_pseudomonas_gene_name_col(summary_gene_ranks, base_dir)
summary_gene_ranks.sort_values(by="Z score", ascending=False).head()

,Gene ID,Adj P-value (Real),Rank (Real),abs(logFC) (Real),logFC (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(logFC) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score,Gene Name
PA4726,PA4726,0.000010,5549.0,4.530917,-4.530917,0.846422,4845.5,87.319755,0.0,0.16708,0.101184,25,43.127565,cbrB
PA1123,PA1123,0.002578,5537.0,2.175900,-2.175900,0.682483,4391.5,79.136626,0.0,0.16190,0.106530,25,18.905485,NaN
PA4620,PA4620,0.001107,5512.0,1.611311,1.611311,0.842271,2220.5,40.005407,0.0,0.11220,0.092593,25,16.190288,NaN
PA1217,PA1217,0.001039,5531.0,1.941753,1.941753,0.850699,1612.0,29.037491,0.0,0.11886,0.120587,25,15.116766,NaN
PA1216,PA1216,0.000223,5545.0,2.600594,2.600594,0.832988,3823.5,68.898702,0.0,0.19604,0.159747,25,15.052271,NaN


In [14]:
summary_gene_ranks.sort_values(by="Percentile (simulated)", ascending=False).head()

,Gene ID,Adj P-value (Real),Rank (Real),abs(logFC) (Real),logFC (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(logFC) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score,Gene Name
PA3049,PA3049,0.012724,5469.0,1.256109,1.256109,0.437968,5549.0,100.000000,0.00,0.73748,0.557901,25,0.929606,rmf
PA2746,PA2746,0.015399,5453.0,1.196825,1.196825,0.407384,5548.0,99.981975,0.08,0.68792,0.380676,25,1.336846,NaN
PA5460,PA5460,0.620038,3121.0,0.219013,0.219013,0.556046,5547.0,99.963951,0.04,0.59978,0.308654,25,-1.233639,NaN
PA2747,PA2747,0.001593,5525.0,1.776658,1.776658,0.474022,5546.0,99.945926,0.00,0.53270,0.265057,25,4.693165,NaN
PA1471,PA1471,0.896482,946.0,0.058927,-0.058927,0.492811,5545.0,99.927902,0.00,0.66966,0.433974,25,-1.407302,NaN


In [15]:
summary_gene_ranks[(summary_gene_ranks["Percent DE (simulated)"] > 0).values]

,Gene ID,Adj P-value (Real),Rank (Real),abs(logFC) (Real),logFC (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(logFC) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score,Gene Name
PA2746,PA2746,0.015399,5453.0,1.196825,1.196825,0.407384,5548.0,99.981975,0.08,0.68792,0.380676,25,1.336846,NaN
PA1176,PA1176,0.028836,5447.0,1.172084,1.172084,0.492811,5520.0,99.477289,0.04,0.46826,0.264577,25,2.660188,napF
PA1190,PA1190,0.013935,5444.0,1.163716,1.163716,0.492811,5541.0,99.855804,0.08,0.57268,0.431481,25,1.369786,NaN
PA1728,PA1728,0.016987,5376.0,0.928531,0.928531,0.393633,5517.0,99.423216,0.04,0.52832,0.332238,25,1.204593,NaN
PA1177,PA1177,0.049481,5261.0,0.717723,0.717723,0.433951,5531.0,99.675559,0.04,0.52524,0.326061,25,0.590327,napE
PA2486,PA2486,0.090075,5134.0,0.610265,0.610265,0.598263,5482.0,98.792358,0.04,0.60242,0.563441,25,0.013923,NaN
PA0106,PA0106,0.264602,5069.0,0.569541,0.569541,0.535868,5530.0,99.657534,0.04,0.47782,0.306917,25,0.298845,coxA
PA4306,PA4306,0.181985,4972.0,0.526076,0.526076,0.492811,5508.0,99.260995,0.08,0.50882,0.371943,25,0.046395,flp
PA5460,PA5460,0.620038,3121.0,0.219013,0.219013,0.556046,5547.0,99.963951,0.04,0.59978,0.308654,25,-1.233639,NaN


In [16]:
# Check if there is an NaN values, there should not be
summary_gene_ranks.isna().any()

Gene ID                              False
Adj P-value (Real)                   False
Rank (Real)                          False
abs(logFC) (Real)                    False
logFC (Real)                         False
Median adj p-value (simulated)       False
Rank (simulated)                     False
Percentile (simulated)               False
Percent DE (simulated)               False
Mean abs(logFC) (simulated)          False
Std deviation (simulated)            False
Number of experiments (simulated)    False
Z score                              False
Gene Name                             True
dtype: bool

In [17]:
# Create `gene_summary_filename`
summary_gene_ranks.to_csv(gene_summary_filename, sep="\t")

## Compare gene ranking

r = 0.44880008139441796
p = 2.8537129367217363e-273
ci_low = 0.42742051598701103
ci_high = 0.4714351366933119


/home/alexandra/anaconda3/envs/generic_expression_new/lib/python3.7/site-packages/matplotlib/font_manager.py:1238: UserWarning: findfont: Font family ['Verdana'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))


In [18]:
# Get generic genes identified by Crow et. al.
GAPE_filename = params["reference_gene_filename"]
ref_gene_col = params["reference_gene_name_col"]
ref_rank_col = params["reference_rank_col"]

figure_filename = f"gene_ranking_{col_to_rank_genes}.svg"

corr, shared_ranking = ranking.compare_gene_ranking(
    summary_gene_ranks, GAPE_filename, ref_gene_col, ref_rank_col, figure_filename
)

In [19]:
# Get genes that are highly generic in both
generic_both = shared_ranking[
    (shared_ranking["Percentile (simulated)"] > 80) & (shared_ranking["prop DEGs"] > 60)
]
generic_both.to_csv(os.path.join(local_dir, "SOPHIE_GAPE_generic.tsv"), sep="\t")

In [20]:
# Get genes that are highly generic by SOPHIE but not by GAPE
generic_SOPHIE_only = shared_ranking[
    (shared_ranking["Percentile (simulated)"] > 80) & (shared_ranking["prop DEGs"] < 40)
]
generic_SOPHIE_only.to_csv(os.path.join(local_dir, "SOPHIE_generic_only.tsv"), sep="\t")

In [21]:
shared_ranking.head()

,Percentile (simulated),prop DEGs,gene id
ProbeID,,,
PA2832_tpm_at,30.064888,44.669913,PA2832
PA2757_at,45.340663,50.279582,PA2757
PA1244_at,95.737203,81.935426,PA1244
PA1500_at,49.684571,1.424964,PA1500
PA4541_at,29.416006,3.760823,PA4541


Hypergeometric test:

Given $N$ number of genes with $K$ common genes in GAPE. SOPHIE identifies $n$ genes as being common. What is the probability that $k$ of the genes identified by SOPHIE are also common in GAPE? What is the probability of drawing $k$ or more concordant genes?

This was a way for us to quantify the correlation between SOPHIE and GAPE common findings, since the correlation coefficient wasn't very convincing since we're considering all genes in addition to the common ones

In [22]:
num_GAPE_genes = shared_ranking.shape[0]
num_generic_GAPE_genes = shared_ranking[
    shared_ranking[ref_rank_col] >= percentile_threshold
].shape[0]
num_generic_SOPHIE_genes = shared_ranking[
    shared_ranking["Percentile (simulated)"] >= percentile_threshold
].shape[0]
num_concordant_generic_genes = shared_ranking[
    (shared_ranking[ref_rank_col] >= percentile_threshold)
    & (shared_ranking["Percentile (simulated)"] >= percentile_threshold)
].shape[0]

In [23]:
print(num_GAPE_genes)
print(num_generic_GAPE_genes)
print(num_generic_SOPHIE_genes)
print(num_concordant_generic_genes)

5549
1084
1110
537


In [24]:
p = ss.hypergeom.sf(
    num_concordant_generic_genes,
    num_GAPE_genes,
    num_generic_GAPE_genes,
    num_generic_SOPHIE_genes,
)
print(p)

1.2247306830592874e-139


**Takeaway:**

* X-axis: gene ranking using SOPHIE (trained on Pseudomonas compendium containing ~1K experiments)
* Y-axis: gene ranking using GAPE (curated set of 73 experiments)

* Overall there is good consistency between SOPHIE and the reference set of experiments. There is especially more consistency amongst lowly ranked genes (genes that consistently didn’t change or genes that changed in a subset of cases). Perhaps the signal from these low ranked genes are very robust in P. aeruginosa
  * Housekeeping genes make sense as contributing to a very strong consistent signal.
  * What about those inconsistently DE genes? Assuming the 73 contexts are represented in the compendium, then those genes that are in consisten
* There is some noise in the bottom right corner (i.e. genes that the reference didn't think were as generic but SOPHIE did). These might be the result of the reference being limited to 73 experiments and differences in data processing (ANOVA vs RMA).
  * Assuming the 73 contexts are represented in the compendium, if a gene is generic in the compendium, then this gene was found to be DE across many contexts, including the subset of 73 contexts. We suspect the reason for generic genes being highly generic in the compendium but NOT in the 73 experiments is because the signal to noise ratio will be lower in the smaller subset that reduce the generic signal.

### GSEA
**Goal:** To detect modest but coordinated changes in prespecified sets of related genes (i.e. those genes in the same pathway or share the same GO term).

1. Ranks all genes based using DE association statistics.
2. An enrichment score (ES) is defined as the maximum distance from the middle of the ranked list. Thus, the enrichment score indicates whether the genes contained in a gene set are clustered towards the beginning or the end of the ranked list (indicating a correlation with change in expression).
3. Estimate the statistical significance of the ES by a phenotypic-based permutation test in order to produce a null distribution for the ES( i.e. scores based on permuted phenotype)

In [25]:
# Create "<local_dir>/GSEA_stats/" subdirectory
os.makedirs(os.path.join(local_dir, "GSA_stats"), exist_ok=True)

In [26]:
# Load pathway data
adage_kegg_DB_filename = params["pathway_DB_filename"]

In [27]:
# Need to format data into tab-delimited matrix
# with columns= KEGG pathway name, description, gene ids
# Each gene ids is tab separated
adage_kegg_DB_processed_filename = os.path.join(
    base_dir, dataset_name, "data", "metadata", "adage_kegg_DB_process_filename.gmt"
)
stats.format_pseudomonas_pathway_DB(
    adage_kegg_DB_filename, local_dir, adage_kegg_DB_processed_filename
)

In [28]:
%%R -i base_dir -i template_DE_stats_filename -i adage_kegg_DB_processed_filename -i statistic -o template_enriched_pathways

source(paste0(base_dir, '/generic_expression_patterns_modules/GSEA_analysis.R'))

out_filename <- paste(local_dir,
                     "GSA_stats/GSEA_stats_template_data_",
                     project_id,
                     "_real.txt",
                     sep = "")

template_enriched_pathways <- find_enriched_pathways(template_DE_stats_filename, adage_kegg_DB_processed_filename, statistic)

template_enriched_pathways <- as.data.frame(template_enriched_pathways[1:7])

write.table(template_enriched_pathways, file = out_filename, row.names = F, sep = "\t")

R[write to console]: Loading required package: Rcpp



In [29]:
print(template_enriched_pathways.shape)
template_enriched_pathways[template_enriched_pathways["padj"] < 0.05].sort_values(
    by="padj"
).head()

(169, 7)


,pathway,pval,padj,ES,NES,nMoreExtreme,size
169,"KEGG-Module-M00178: Ribosome, bacteria",0.000112,0.001024,0.914364,6.153462,0.0,55
75,KEGG-Pathway-pae00680: Methane metabolism - Ps...,0.000127,0.001024,0.550329,2.743014,0.0,24
62,KEGG-Pathway-pae00970: Aminoacyl-tRNA biosynth...,0.000126,0.001024,0.669514,3.432885,0.0,26
60,KEGG-Pathway-pae00240: Pyrimidine metabolism -...,0.000116,0.001024,0.501650,3.106829,0.0,43
52,KEGG-Pathway-pae00620: Pyruvate metabolism - P...,0.000112,0.001024,0.366825,2.504805,0.0,58


In [30]:
%%R -i project_id -i local_dir -i adage_kegg_DB_processed_filename -i num_runs -i statistic

source(paste0(base_dir, '/generic_expression_patterns_modules/GSEA_analysis.R'))

# New files created: "<local_dir>/GSEA_stats/GSEA_stats_simulated_data_<project_id>_<n>.txt"
for (i in 0:(num_runs-1)) {
    simulated_DE_stats_filename <- paste(local_dir,
                                     "DE_stats/DE_stats_simulated_data_",
                                     project_id,
                                     "_",
                                     i,
                                     ".txt",
                                     sep = "")

    out_filename <- paste(local_dir,
                     "GSA_stats/GSEA_stats_simulated_data_",
                     project_id,
                     "_",
                     i,
                     ".txt",
                     sep = "")

    enriched_pathways <- find_enriched_pathways(simulated_DE_stats_filename, adage_kegg_DB_processed_filename, statistic)

    # Remove column with leading edge since its causing parsing issues
    write.table(as.data.frame(enriched_pathways[1:7]), file = out_filename, row.names = F, sep = "\t")
}

### Rank pathways

In [31]:
analysis_type = "GSA"
template_GSEA_stats_filename = os.path.join(
    local_dir, "GSA_stats", f"GSEA_stats_template_data_{project_id}_real.txt"
)
(
    template_GSEA_stats,
    simulated_GSEA_summary_stats,
) = ranking.process_and_rank_genes_pathways(
    template_GSEA_stats_filename,
    local_dir,
    num_runs,
    project_id,
    analysis_type,
    col_to_rank_pathways,
    logFC_name,
    pvalue_name,
    "GSEA",
)

### Pathway summary table

In [32]:
# Create intermediate file: "<local_dir>/gene_summary_table_<col_to_rank_pathways>.tsv"
summary_pathway_ranks = ranking.generate_summary_table(
    template_GSEA_stats_filename,
    template_GSEA_stats,
    simulated_GSEA_summary_stats,
    col_to_rank_pathways,
    local_dir,
    "pathway",
    params,
)

summary_pathway_ranks.sort_values(by="Rank (simulated)", ascending=False).head()

/home/alexandra/anaconda3/envs/generic_expression_new/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,Pathway ID,Adj P-value (Real),Rank (Real),padj (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Mean padj (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
pathway,,,,,,,,,,,
"KEGG-Module-M00178: Ribosome, bacteria","KEGG-Module-M00178: Ribosome, bacteria",0.001024,158.0,0.001024,0.001232,161.5,100.0,0.001306,0.000290,25,-0.971022
KEGG-Pathway-pae00970: Aminoacyl-tRNA biosynthesis - Pseudomonas aeruginosa PAO1,KEGG-Pathway-pae00970: Aminoacyl-tRNA biosynth...,0.001024,158.0,0.001024,0.001232,161.5,100.0,0.001306,0.000290,25,-0.971022
"KEGG-Module-M00157: F-type ATPase, prokaryotes and chloroplasts","KEGG-Module-M00157: F-type ATPase, prokaryotes...",0.001034,145.0,0.001034,0.001232,161.5,100.0,0.001348,0.000396,25,-0.792923
KEGG-Module-M00335: Sec (secretion) system,KEGG-Module-M00335: Sec (secretion) system,0.001032,146.0,0.001032,0.001232,161.5,100.0,0.001545,0.001294,25,-0.396777
"KEGG-Module-M00011: Citrate cycle, second carbon oxidation, 2-oxoglutarate => oxaloacetate","KEGG-Module-M00011: Citrate cycle, second carb...",0.001024,158.0,0.001024,0.001232,161.5,100.0,0.001369,0.000436,25,-0.790494


In [33]:
# Create `pathway_summary_filename`
summary_pathway_ranks.to_csv(pathway_summary_filename, sep="\t")

**Quick check:**

Looks like Ribosomes are found to be significantly differentiated in the template and across 25 simulated experiments. So this pathway looks to be generic, which is consistent with findings from [Crow. et. al.](https://www.pnas.org/content/116/13/6491.abstract), using Human data. This is also consistent with [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2679180/) which observed that Ribosome proteins have many different functions in *S. cerevisiae*, which might explain their variability across a wide range of contexts.

* Need to think about why it might make sense that Ribosomes being generic in Humans would also imply that they are generic in P. aeruginosa.